```
Copyright (c) Gradient Institute. All rights reserved.
Licensed under the Apache 2.0 License.
```


This notebook tests out this fix for estimating treatment effect in regularised linear models:

Hahn, P.R., Carvalho, C.M., Puelz, D., He, J., 2018. Regularization and Confounding in Linear Regression for Treatment Effect Estimation. Bayesian Anal. 13. https://doi.org/10.1214/16-BA1044


In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, BayesianRidge, Lasso, Ridge, ARDRegression
from sklearn.inspection import plot_partial_dependence
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score

from twostageridge import TwoStageRidge

# Generate the data

In [2]:
N = 500
D_rand = 10  # concatenate this many random columns onto X
random_proj = True  # randomly project control variables
proj_D = 50

# Confounders - X
mu_x = np.zeros(4)
cov_x = np.array([
    [ 1.0,  0.5,  0.0,  0.3],
    [ 0.5,  1.0, -0.2,  0.0],
    [ 0.0, -0.2,  1.0, -0.7],
    [ 0.3,  0.0, -0.7,  1.0]
])


# Treatment - Z
gamma = np.array([2., 1., -0.7, -0.5])
sig_eps = 0.2


# Target - Y
alpha = 0.5
beta = np.array([1., -2., 1.5, 0.6])
sig_nu = 0.9

# Generation function
def generate_data():
    X = np.random.multivariate_normal(mean=mu_x, cov=cov_x, size=N)
    Z = X @ gamma + sig_eps * np.random.randn(N)
    Y = alpha * Z + X @ beta + sig_nu * np.random.randn(N)

    X = np.hstack((X, np.random.randn(N, D_rand)))
    if random_proj:
        D = X.shape[1]
        X = X @ np.random.standard_t(size=(D, proj_D), df=5)

    W = np.hstack((Z[:, np.newaxis], X))
    
    return W, Y

# Generate bias comparison results

In [3]:
S = 20
models = {
    "ols": LinearRegression(),
    "lasso": Lasso(alpha=1e-2, max_iter=20000),
    "ridge": Ridge(alpha=1.),
    "bayes": BayesianRidge(),
    #"ard": ARDRegression(),
    "ts": TwoStageRidge(treatment_index=0, regulariser1=1., regulariser2=1.)
}

scores = {k: [] for k in models.keys()}
biases = {k: [] for k in models.keys()}
errors = {k: [] for k in models.keys()}

for s in range(S):
    W, Y = generate_data()
    
    for name, model in models.items():
        model.fit(W, Y)
        alpha_hat = model.alpha_ if name == "ts" else model.coef_[0] 
        bias = np.abs(alpha - alpha_hat)
        error = bias**2
        
        scores[name].append(model.score(W, Y))
        biases[name].append(bias)
        errors[name].append(error)
        
    print('.', end='')
print()
        
        
scores = {k: np.mean(v) for k, v in scores.items()}
biases = {k: np.mean(v) for k, v in biases.items()}
errors = {k: np.sqrt(np.mean(v)) for k, v in errors.items()}


results = pd.DataFrame({
    "scores": scores,
    "biases": biases,
    "errors": errors
})

results

....................


,scores,biases,errors
ols,0.866871,0.147680,0.180844
lasso,0.866498,0.270546,0.314848
ridge,0.866927,0.142757,0.176707
bayes,0.865899,0.392242,0.395786
ts,0.866932,0.147153,0.179803


In [4]:
models["ts"].score_stage1(W)

0.992976563350982